In [ ]:
# For google colab
#!pip install erddap-python

In [11]:
import matplotlib.pyplot as plt
from erddapClient import ERDDAP_Griddap
import pandas as pd
from pathlib import Path
import imageio
from tqdm import tqdm
import xarray as xr
import warnings
from loguru import logger
from glob import glob
from hakai_salish_sea_model_tools.salishseacast import NEMOGrid

import matplotlib

matplotlib.use("Agg")

warnings.simplefilter(action="ignore", category=FutureWarning)

## Define reference stations

In [15]:
stations = [
    {
        "name": "QU5",
        "latitude": 50.1183,
        "longitude":-125.2122,
    },{
        "name": "QU39",
        "latitude": 50.0307000000001,
        "longitude":-125.0992,
    },
    {
        "name": "W1",
        "latitude": 50.088,
        "longitude":-125.111,
    },{
        "name": "W2",
        "latitude": 49.989,
        "longitude":-125.060,
    },{
        "name": "Sentry Shoal",
        "latitude": 49.92,
        "longitude":-125.0,
    }
]

# Find their corresponding node on the NEMO grid
neamodel = NEMOGrid()
for station in stations:
    grid_point = neamodel.ll2grid(station["latitude"], station["longitude"],2,output="dict")
    station["gridX"] = grid_point["gridX"]
    station["gridY"] = grid_point["gridY"]
    station['gridLatitude'] = grid_point['latitude']
    station['gridLongitude'] = grid_point['longitude']
    station["distance_from_node"] = grid_point["distance"]
df_stations = pd.DataFrame(stations)
df_stations

,name,latitude,longitude,gridX,gridY,gridLatitude,gridLongitude,distance_from_node
0,QU5,50.1183,-125.2122,138,761,50.118435,-125.215340,0.224355
1,QU39,50.0307,-125.0992,144,736,50.032028,-125.101738,0.233828
2,W1,50.0880,-125.1110,150,748,50.090042,-125.110069,0.236582
3,W2,49.9890,-125.0600,145,725,49.990959,-125.060219,0.218410
4,Sentry Shoal,49.9200,-125.0000,145,707,49.920689,-125.000687,0.090994


In [16]:
@logger.catch
def get_data(server, datasetID, vars, time, nc_output_file):
    subset = (
        ERDDAP_Griddap(server, datasetID)
        .setResultVariables(list(vars.keys()))
        .setSubset(
            time=time,
            depth=slice(0.5000003, 0.5000003),
            gridY=slice(600, 897),
            gridX=slice(100, 300),
        )
        .getxArray()
    )
    # save dataset
    # Drop some atttributes that fails to be encoded
    for var in subset.variables:
        subset[var].attrs.pop("_evenlySpaced", None)
        subset[var].attrs.pop("actual_range", None)
        subset[var].attrs.pop("units", None)
        subset[var].attrs.pop("calendar", None)
    subset.to_netcdf(nc_output_file)
    return subset

In [4]:
@logger.catch
def get_frame(time, vars, output, server, datasetID):
    """Get water properties frames"""
    nc_output_file = Path(output.format(time=time, var="-".join(vars.keys())) + ".nc")
    if nc_output_file.exists():
        subset = xr.open_dataset(nc_output_file)
    else:
        subset = get_data(server, datasetID, vars, time, nc_output_file)

    for var, attrs in vars.items():
        _, ax = plt.subplots()
        subset[var].plot(ax=ax, **attrs)
        plt.savefig(output.format(time=time, var=var))

## Time Range of Interest


In [24]:
# Ben's paper 2016
time_frames = pd.date_range(
    start=pd.to_datetime("2016-07-25T00:30:00Z"),
    end=pd.to_datetime("2016-08-05T00:30:00Z"),
    freq=pd.Timedelta("1h"),
)
output = Path("output")
output.mkdir(parents=True, exist_ok=True)

In [6]:
# Small wind event 2016
time_frames = pd.date_range(
    start=pd.to_datetime("2023-06-12T00:30:00Z"),
    end=pd.to_datetime("2023-06-18T00:30:00Z"),
    freq=pd.Timedelta("1h"),
)
ouput = Path("output_2023-06-12")
output.mkdir(parents=True, exist_ok=True)

## Water Properties 

In [7]:
# Get temperature,salinity frames
server = "https://salishsea.eos.ubc.ca/erddap"
datasetID = "ubcSSg3DPhysicsFields1hV21-11"

vars = {
    "temperature": {"vmin": 5, "vmax": 25, "cmap": "inferno"},
    "salinity": {"vmin": 20, "vmax": 32, "cmap": "viridis"},
}

data = []
for time in tqdm(time_frames, desc="Generate frames", unit="time_stamp"):
    get_frame(
        time.isoformat().replace("+00:00", "Z"),
        vars,
        "output/{var}_{time}",
        server,
        datasetID,
    )

Generate frames:   7%|▋         | 10/145 [00:53<11:51,  5.27s/time_stamp]/var/folders/bv/k82zg3_x1fg00tjywdqys88h0000gn/T/ipykernel_85026/2377147092.py:11: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  _, ax = plt.subplots()
Generate frames: 100%|██████████| 145/145 [13:14<00:00,  5.48s/time_stamp]


## Load Current data

In [8]:
subset = (
    ERDDAP_Griddap("https://salishsea.eos.ubc.ca/erddap", "ubcSSg3DuGridFields1hV21-11")
    .setResultVariables("uVelocity")
    .setSubset(
        time=time,
        depth=slice(0.5000003, 0.5000003),
        gridY=slice(600, 897),
        gridX=slice(100, 300),
    )
    .getxArray()
)

In [9]:
server = "https://salishsea.eos.ubc.ca/erddap"
datasetIDs = {
    "uVelocity": "ubcSSg3DuGridFields1hV21-11",
    "vVelocity": "ubcSSg3DvGridFields1hV21-11",
}

for time in tqdm(time_frames, desc="Generate frames", unit="time_stamp"):
    for var, datasetID in datasetIDs.items():
        get_frame(
            time.isoformat().replace("+00:00", "Z"),
            {var: {"vmin": -1, "vmax": 1, "cmap": "coolwarm"}},
            "output/{var}_{time}",
            server,
            datasetID,
        )

Generate frames: 100%|██████████| 145/145 [20:23<00:00,  8.44s/time_stamp]


## Load Sentry Shoal

In [9]:
## Load Sentry Shoal data

## Generate Combined figures

In [25]:
# Generate each frames
def make_combined_figure(timestamp_str):
    try:
        ds_water = xr.open_dataset(f"output/temperature-salinity_{timestamp_str}.nc")
        ds_u = xr.open_dataset(f"output/uVelocity_{timestamp_str}.nc")
        ds_v = xr.open_dataset(f"output/vVelocity_{timestamp_str}.nc")
    except:
        logger.error(f"Could not open files for {timestamp_str}")
        return
    
    for var in ["temperature", "salinity"]:
        ds_water[var] = ds_water[var].where(ds_water[var] > 0)
    
    ds_u["uVelocity"] = ds_u["uVelocity"].where(ds_u["uVelocity"] != 0)
    ds_v["vVelocity"] = ds_v["vVelocity"].where(ds_v["vVelocity"] != 0)

    plt.close("all")

    def get_cmap(cmap_name):
        cmap = plt.get_cmap(cmap_name)
        cmap.set_under(color="white")
        return cmap

    cur_spd = (ds_u.uVelocity**2 + ds_v.vVelocity**2) ** (1 / 2)

    col = 2
    row = 2
    fig, axs = plt.subplots(col, row, figsize=(8, 5))
    fig.suptitle(f"SalishSeaCast Model Output: {timestamp_str}")
    ds_water.temperature.plot(ax=axs[0, 0], cmap="turbo", vmin=12, vmax=22)
    ds_water.salinity.plot(ax=axs[0, 1], cmap="turbo", vmin=24, vmax=30)
    ((ds_u.uVelocity**2 + ds_v.vVelocity**2) ** (1 / 2)).plot(
        ax=axs[1, 0],
        cmap=get_cmap("turbo"),
        vmin=0.01,
        vmax=1,
        cbar_kwargs={"label": "Current Speed (m/s)"},
    )
    plt.quiver(
        ds_u.gridX,
        ds_u.gridY,
        ds_u.uVelocity.isel(time=0, depth=0),
        ds_v.vVelocity.isel(time=0, depth=0),
        cur_spd.where(cur_spd > 0.1),
        scale=100,
        cmap="turbo",
    )
    # add stations on each axes
    for ax in axs.flatten():
        for station in stations:
            ax.plot(
                station["gridX"],
                station["gridY"],
                "x",
                color="red",
                markersize=2,
                label=station["name"],
            )
    # ds_v.vVelocity.plot(ax=axs[1, 1], cmap='coolwarm', vmin=-1, vmax=1)

    for i in range(col):
        for j in range(row):
            axs[i, j].set_title(None)
            axs[i, j].set_xlabel(None)
            axs[i, j].set_ylabel(None)
            axs[i, j].set_xticks([])
            axs[i, j].set_yticks([])

    plt.savefig(
        f"output/combined_{timestamp_str}.png",
        dpi=300,
        bbox_inches="tight",
        pad_inches=0.1,
    )

In [26]:
for timestamp in tqdm(time_frames, desc="Generate combined frames", unit="frame"):
    timestamp_str = timestamp.isoformat().replace("+00:00", "Z")
    make_combined_figure(timestamp_str)

Generate combined frames: 100%|██████████| 265/265 [04:44<00:00,  1.07s/frame]


In [27]:
def make_gif(files, output, fps):
    frames = []
    for file in tqdm(sorted(glob(files)), desc="Load frames", unit="frame"):
        image = imageio.v2.imread(file)
        frames.append(image)
    logger.info(f"Saving gif to {output}")
    imageio.mimsave(output, frames, fps=fps)
    logger.info(f"Done")

In [28]:
make_gif("output/combined_2016-*.png", "../docs/NSOG_animation_20160725-20160805.gif", 4)
# make_gif("output/combined_2023-*.png", "../docs/NSOG_animation_20230612-20230618.gif", 4)

Load frames: 100%|██████████| 265/265 [00:16<00:00, 16.16frame/s]
2024-02-15 14:22:23.177 | INFO     | __main__:make_gif:6 - Saving gif to ../docs/NSOG_animation_20160725-20160805.gif
2024-02-15 14:22:42.170 | INFO     | __main__:make_gif:8 - Done
